# Tomato Leaf Disease Detection

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importing the Libraries

In [3]:
import tensorflow as tf
import pandas as pd 
from tensorflow.keras.utils import to_categorical
import random
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Activation, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, array_to_img, img_to_array
from tensorflow.keras.models import Sequential
from glob import glob

## Importing the Dataset

In [4]:
# Define Constants by re-sizing all the images
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/Project__Dataset/train'
valid_path = '/content/drive/MyDrive/Project__Dataset/valid'

#### Inception V3

In [5]:
# Import the InceptionV3 model and here we will be using imagenet weights

inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

87924736/87910968 [==============================] - 1s 0us/step


In [6]:
# We don't need to train existing weights
for layer in inception.layers:
    layer.trainable = False

In [7]:
# Folders in the Training Set
folders = glob('/content/drive/MyDrive/Project__Dataset/train/*')
folders

['/content/drive/MyDrive/Project__Dataset/train/Potato___Late_blight',
 '/content/drive/MyDrive/Project__Dataset/train/Potato___Early_blight',
 '/content/drive/MyDrive/Project__Dataset/train/Potato___healthy']

In [8]:
# Model layers -> can add more if required
x = Flatten()(inception.output)
prediction = Dense(len(folders), activation='softmax')(x)

In [9]:
# Create a model object
model = Model(inputs=inception.input, outputs=prediction)

# View the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [10]:
# Defining the cost and model optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [11]:
# Using the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [17]:
# Training Generator
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Project__Dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 64,
                                                 class_mode = 'categorical')

Found 1300 images belonging to 3 classes.


In [18]:
# Testing Generator
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Project__Dataset/valid',
                                            target_size = (224, 224),
                                            batch_size = 64,
                                            class_mode = 'categorical')

Found 852 images belonging to 3 classes.


In [19]:
# fit the model, it will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
21/21 [==============================] - 393s 19s/step - loss: 1.0753 - accuracy: 0.8692 - val_loss: 0.6260 - val_accuracy: 0.8991
Epoch 2/10
21/21 [==============================] - 291s 14s/step - loss: 0.4622 - accuracy: 0.9246 - val_loss: 0.4429 - val_accuracy: 0.9155
Epoch 3/10
21/21 [==============================] - 288s 14s/step - loss: 0.2271 - accuracy: 0.9508 - val_loss: 0.3726 - val_accuracy: 0.9319
Epoch 4/10
21/21 [==============================] - 236s 11s/step - loss: 0.2268 - accuracy: 0.9523 - val_loss: 0.3649 - val_accuracy: 0.9354
Epoch 5/10
21/21 [==============================] - 291s 14s/step - loss: 0.1110 - accuracy: 0.9662 - val_loss: 0.3119 - val_accuracy: 0.9437
Epoch 6/10
21/21 [==============================] - 291s 14s/step - loss: 0.0882 - accuracy: 0.9692 - val_loss: 0.3051 - val_accuracy: 0.9308
Epoch 7/10
21/21 [==============================] - 237s 11s/step - loss: 0.1187 - accuracy: 0.9638 - val_loss: 0.4026 - val_accuracy: 0.9202
Epoch 

## Visualize the model training by plotting Loss Function and Accuracy

In [1]:
# Plot the Loss and Accuracy
# Loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# Accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

NameError: ignored

In [24]:
# Saving the model as a h5 file

from tensorflow.keras.models import load_model
model.save('/content/drive/MyDrive/xx_inception.h5')

In [ ]:
y_pred = model.predict(test_set)
y_pred

array([[2.2761736e-35, 1.0000000e+00, 4.9953767e-30, ..., 0.0000000e+00,
        2.4183147e-32, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 1.0000000e+00],
       [3.5787742e-23, 1.0000000e+00, 3.9931668e-26, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [4.9600962e-31, 0.0000000e+00, 3.7623045e-30, ..., 0.0000000e+00,
        0.0000000e+00, 2.8226938e-26],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [3.9725606e-27, 2.9429740e-17, 2.0605025e-07, ..., 1.0230512e-28,
        1.1873254e-22, 3.3261676e-21]], dtype=float32)

In [ ]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)
y_pred

array([1, 9, 1, ..., 5, 7, 3])